In [5]:
import requests
import datetime
from time import sleep
import csv

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# ブラウザのオプションを格納する変数。
options = Options()
# Headlessモードを有効にする（コメントアウトするとブラウザが実際に立ち上がる）
options.set_headless(True)

/Users/shion/.local/share/virtualenvs/stock_project-KOCIdcQu/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: use setter for headless property instead of set_headless
  from ipykernel import kernelapp as app


In [2]:
data = pd.read_csv('input/N225f_2019.csv')
data.head()

,日付,時間,始値,高値,安値,終値,出来高
0,2019/1/4,16:30,19820,19820,19790,19800,166
1,2019/1/4,16:31,19790,19800,19780,19790,176
2,2019/1/4,16:32,19790,19810,19781,19800,181
3,2019/1/4,16:33,19790,19810,19790,19800,84
4,2019/1/4,16:34,19810,19820,19810,19810,26


In [3]:
# scrape stock prices
# URL = "http://www.nikkei.com/markets/kabu/"
URL = "https://jp.investing.com/indices/japan-225-futures"
def get_stock_price1(url):

    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")
    span = soup.find(class_="mkc-stock_prices")
    
    return span.string.strip(',')

def get_stock_price2(url):
    # ブラウザを起動する
    driver = webdriver.Chrome(chrome_options=options)

    # ブラウザでアクセスする
    driver.get(url)

    # HTMLを文字コードをUTF-8に変換してから取得。
    
    html = driver.page_source.encode('utf-8')
    # BeautifulSoupで扱えるようにパース。
    soup = BeautifulSoup(html, "html.parser")

    return soup.select_one("#last_last").text.strip(',')

In [ ]:
# 特徴量生成


In [16]:
# {'日付': 0, '時間': 1, '始値': 2, '高値': 3, '安値': 4, '終値': 5, '出来高':6}
while (1):
    dt_now = datetime.datetime.now()
    list_data = []
    st_price = get_stock_price2(URL)
    
    list_data[0:2] = [dt_now.strftime('%Y/%m/%d'), dt_now.strftime('%H:%M')]
    list_data[2:-1] = [st_price.replace(',', '') for i in range(4)]
    list_data.append(0.0)

    while(1):
        if dt_now.minute == datetime.datetime.now().minute:
            
            st_price_new =  get_stock_price2(URL).replace(',', '')

            # 高値の更新
            if list_data[3] < st_price_new:
                list_data[3] = st_price_new

            # 安値の更新
            if list_data[4] > st_price_new:
                list_data[4] = st_price_new

            # 終値の更新
            list_data[5] = st_price_new
            
            # sleep
            sleep(5)
        else:
            with open('input/N225f_2019.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(list_data)
            break

/Users/shion/.local/share/virtualenvs/stock_project-KOCIdcQu/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  
/Users/shion/.local/share/virtualenvs/stock_project-KOCIdcQu/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


21482.5 2019-10-08 18:59:08.119581


/Users/shion/.local/share/virtualenvs/stock_project-KOCIdcQu/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  
/Users/shion/.local/share/virtualenvs/stock_project-KOCIdcQu/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


21477.5 2019-10-08 18:59:43.240859


KeyboardInterrupt: 

In [17]:
data = pd.read_csv('input/N225f_2019.csv')
data.tail()

,日付,時間,始値,高値,安値,終値,出来高
184000,2019/8/27,15:12,20430.0,20430.0,20430.0,20430.0,0.0
184001,2019/8/27,15:13,20430.0,20430.0,20430.0,20430.0,0.0
184002,2019/8/27,15:14,20430.0,20430.0,20430.0,20430.0,0.0
184003,2019/8/27,15:15,20420.0,20420.0,20420.0,20420.0,24842.0
184004,2019/10/08,18:58,21482.5,21482.5,21482.5,21482.5,0.0
